In [1]:
import napari
import tifffile
import numpy as np
import h5py

In [2]:
dir = '/Users/hongwang/Documents/Eureka/'

In [14]:
# train seed = 0
# test seed (cur val, 7 files) = 0
# test seed for sigma = 0.1 (cur val, 7 files) = 1000
# test seed for sigma = 0.3 (cur val, 7 files) = 10000
# validation seed for sigma = 0.1 (cur test, 2 files) = 100
# validation seed for sigma = 0.3 (cur test, 2 files) = 10
np.random.seed(0)

## test generating noisy data

In [6]:
img = h5py.File(dir+'ovules_10k/train_noise/N_404_ds2x.h5','r')
print(img.keys())
raw = img['raw']
trueseg = img['label']
imglabelIgnore = img['label_with_ignore']

print('raw type and shape',type(raw),raw.shape)
print('truseg type and shape',type(trueseg),trueseg.shape)
# napari code - this will only run on laptop with napari installed
viewer = napari.view_image(raw,ndisplay=3,blending='additive')
viewer.add_labels(trueseg)
viewer.add_labels(imglabelIgnore)
napari.run()

<KeysViewHDF5 ['label', 'label_with_ignore', 'raw']>
raw type and shape <class 'h5py._hl.dataset.Dataset'> (321, 563, 968)
truseg type and shape <class 'h5py._hl.dataset.Dataset'> (321, 563, 968)


In [7]:
raw_np = np.array(raw)
print(raw_np.min())
print(raw_np.max())
print(np.median(raw_np))
print((raw_np==96).sum())
#print(np.random.normal(0, 0.5, raw_np.shape).shape)
#print(np.random.normal(0, 0.5, raw_np.shape)[:10][:10][:10])

0
255
0.0
207991


In [106]:
print(raw_np.dtype, raw_np.min(), raw_np.max())
raw_max=raw_np.max()
raw_np_norm = raw_np/raw_max
print(raw_np[210][510][510])
print(raw_np_norm[210][510][510])
raw_np_norm += np.random.normal(0, 0.5, raw_np.shape)
print(raw_np_norm.min(), raw_np_norm.max())
raw_np_norm = np.clip(raw_np_norm, 0., 1.)
print(raw_np_norm.min(), raw_np_norm.max())
raw_np_norm *= raw_max
print(raw_np_norm.min(), raw_np_norm.max())
raw_np_noise = np.uint8(raw_np_norm)
print(raw_np_noise.shape, raw_np_noise.min(), raw_np_noise.max())

uint8 0 255
0
0.0
-2.918026607767632 3.759729652210135
0.0 1.0
0.0 255.0
(321, 563, 968) 0 255


In [84]:
viewer = napari.view_image(raw_np_noise,ndisplay=3,blending='additive')
napari.run()

In [84]:
img = h5py.File(dir+'ovules_10k/val/N_464_ds2x.h5','r')
raw = img['raw']
trueseg = img['label']
imglabelIgnore = img['label_with_ignore']

f = h5py.File(dir+'ovules_10k/test.h5', "w")
dset1 = f.create_dataset("raw", data = raw_np_noise)
dset2 = f.create_dataset("label", data = trueseg)
dset3 = f.create_dataset("label_with_ignore", data = imglabelIgnore)
f.close()

img = h5py.File(dir+'ovules_10k/test.h5','r')
raw = img['raw']
trueseg = img['label']
imglabelIgnore = img['label_with_ignore']
img.close()

## generate noisy data

In [35]:
img = h5py.File(dir+'ovules_10k/val/N_294_final_crop_ds2.h5','r')
print(img.keys())
raw = img['raw']
trueseg = img['label']
imglabelIgnore = img['label_with_ignore']
print(raw.shape)
print(type(raw))
print(raw.dtype)
print(trueseg.dtype)
print(imglabelIgnore.dtype)
raw_np = np.array(raw)

<KeysViewHDF5 ['label', 'label_with_ignore', 'raw']>
(320, 960, 1000)
<class 'h5py._hl.dataset.Dataset'>
uint8
uint16
int16


In [ ]:
print(raw_np[:5][:5][:5])
raw_max=raw_np.max()
raw_np_norm = raw_np/raw_max
raw_np_norm += np.random.normal(0, 0.3, raw_np.shape)
raw_np_norm = np.clip(raw_np_norm, 0., 1.)
raw_np_norm *= raw_max
raw_np_noise = np.uint8(raw_np_norm)
print(raw_np_noise[:5][:5][:5])

In [37]:
f = h5py.File(dir+'ovules_10k/val_noise_03/N_294_final_crop_ds2.h5', "w")
dset1 = f.create_dataset("raw", data = raw_np_noise)
dset2 = f.create_dataset("label", data = trueseg)
dset3 = f.create_dataset("label_with_ignore", data = imglabelIgnore)
f.close()

In [38]:
img = h5py.File(dir+'ovules_10k/val_noise_03/N_294_final_crop_ds2.h5','r')
print(img.keys())
raw = img['raw']
print(raw.dtype)
print(type(raw))
print(raw.shape)
trueseg = img['label']
imglabelIgnore = img['label_with_ignore']
print(trueseg.dtype)
print(imglabelIgnore.dtype)
print(np.array(raw)[:5][:5][:5])
img.close()

<KeysViewHDF5 ['label', 'label_with_ignore', 'raw']>
uint8
<class 'h5py._hl.dataset.Dataset'>
(320, 960, 1000)
uint16
int16
[[[  0  13   0 ...   0  48   0]
  [  0   0   0 ...  58   0   0]
  [102   0   0 ...  73   0  40]
  ...
  [ 95  76   0 ...   0  17   0]
  [  0   0   0 ...   0   0 103]
  [194  22   0 ...   5   0   0]]

 [[153  43  16 ...   0 115   0]
  [ 44  15  11 ...   0   0   0]
  [  0   0  21 ...  44   0   0]
  ...
  [ 45 168 153 ...  51   7  26]
  [  0   0  28 ...   0   0  92]
  [  0   0 137 ...  83  58  21]]

 [[  0   0   0 ...   0   0  75]
  [191  48  47 ...  45   0  77]
  [156   0   0 ...   0   0   9]
  ...
  [130   5   0 ...  85  48   0]
  [  9   0   0 ...   0  57   0]
  [  0   0   0 ...  90   0  40]]

 [[  0 137   0 ... 143   0  72]
  [  0  59   0 ...  80  78 208]
  [ 94  37  51 ...   0  57  64]
  ...
  [  0   0  23 ...   0 106  36]
  [ 56   0   0 ...   0   0  10]
  [ 77   0   0 ...   0  55   0]]

 [[  0   0  16 ...  57   0   0]
  [  0  64  50 ...   0   0   0]
  [136   0  

## blur 3D image

In [4]:
from scipy.ndimage import gaussian_filter
import os

In [13]:
img = h5py.File(dir+'ovules_10k/val/N_294_final_crop_ds2.h5','r')
print(img.keys())
raw = img['raw']
trueseg = img['label']
imglabelIgnore = img['label_with_ignore']
print(raw.shape)
print(type(raw))
print(raw.dtype)
print(trueseg.dtype)
print(imglabelIgnore.dtype)
raw_np = np.array(raw)

<KeysViewHDF5 ['label', 'label_with_ignore', 'raw']>
(320, 960, 1000)
<class 'h5py._hl.dataset.Dataset'>
uint8
uint16
int16


In [14]:
viewer = napari.view_image(raw,ndisplay=3,blending='additive')
viewer.add_labels(trueseg)
viewer.add_labels(imglabelIgnore)
napari.run()

In [24]:
print(raw_np[:5][:5][:5])
raw_max=raw_np.max()
raw_np_norm = raw_np/raw_max
raw_np_norm = gaussian_filter(raw_np_norm, sigma = 5.)
raw_np_norm += np.random.normal(0, 0.5, raw_np.shape)
raw_np_norm = np.clip(raw_np_norm, 0., 1.)
raw_np_norm *= raw_max
raw_np_noise = np.uint8(raw_np_norm)
#print(raw_np_noise[:5][:5][:5])

[[[ 0  0  0 ...  0  0  0]
  [ 0  0  0 ...  0  0  0]
  [ 0  0  0 ...  0  0  0]
  ...
  [ 0  0  0 ...  0  0  0]
  [ 0  0  0 ...  0  0  0]
  [ 0  0  0 ...  0  0  0]]

 [[ 0  0  0 ...  0  0  0]
  [ 0  0  0 ...  0  0  0]
  [ 0  0  0 ...  0  0  0]
  ...
  [ 0  0  0 ...  0  0  0]
  [ 0  0  0 ...  0  0  0]
  [ 0  0  0 ...  0  0  0]]

 [[ 0  0  0 ...  0  0  0]
  [ 0  0  0 ...  0  0  0]
  [ 0  0  0 ...  0  0  0]
  ...
  [ 0  0  0 ...  0  0  0]
  [14  0  0 ...  0  0  0]
  [ 0  0  0 ...  0  0  0]]

 [[ 0  0  0 ...  0  0  0]
  [ 0  0  0 ...  0  0  0]
  [ 0  0  0 ...  0  0  0]
  ...
  [ 0  0  0 ...  0  0  0]
  [ 0  0  0 ...  0  0  0]
  [ 0  0  0 ...  0  0  0]]

 [[ 0  0  0 ...  0  0  0]
  [ 0  0  0 ...  0  0  0]
  [ 0  0  0 ...  0  0  0]
  ...
  [ 0  0  0 ...  0  0  0]
  [ 0  0  0 ...  0  0  0]
  [ 0  0  0 ...  0  0  0]]]


In [25]:
viewer = napari.view_image(raw_np_noise,ndisplay=3,blending='additive')
napari.run()

In [ ]:
f = h5py.File(dir+'ovules_10k/val_noise_03/N_294_final_crop_ds2.h5', "w")
dset1 = f.create_dataset("raw", data = raw_np_noise)
dset2 = f.create_dataset("label", data = trueseg)
dset3 = f.create_dataset("label_with_ignore", data = imglabelIgnore)
f.close()

In [17]:
files = os.listdir(dir+'ovules_10k/test')
files.sort()
print(files)
for file in files:
    if file[-2:] == 'h5':
        img = h5py.File(dir+'ovules_10k/test/'+file,'r')
        raw = img['raw']
        trueseg = img['label']
        imglabelIgnore = img['label_with_ignore']
        raw_np = np.array(raw)
        
        raw_max=raw_np.max()
        raw_np_norm = raw_np/raw_max
        raw_np_norm = gaussian_filter(raw_np_norm, sigma = 10.)
        raw_np_norm += np.random.normal(0, 0.5, raw_np.shape)
        raw_np_norm = np.clip(raw_np_norm, 0., 1.)
        raw_np_norm *= raw_max
        raw_np_noise = np.uint8(raw_np_norm)
        
        f = h5py.File(dir+'ovules_10k/test_blur10/'+file, "w")
        dset1 = f.create_dataset("raw", data = raw_np_noise)
        dset2 = f.create_dataset("label", data = trueseg)
        dset3 = f.create_dataset("label_with_ignore", data = imglabelIgnore)
        f.close()

['.DS_Store', 'N_420_ds2x.h5', 'N_420_ds2x.tif', 'N_464_ds2x.h5', 'N_464_ds2x.tif', 'PreProcessing_noise-test-orig', 'PreProcessing_orig-test-orig', 'index.html?zip=']


In [6]:
img = h5py.File(dir+'ovules/PreProcessing/CHECKPOINT_DIR_ovules/N_294_final_crop_ds2_predictions.h5','r')
print(img.keys())
predictions = img['predictions']
viewer = napari.view_image(predictions,ndisplay=3,blending='additive')
#viewer.add_labels(trueseg)
napari.run()

<KeysViewHDF5 ['predictions']>


In [8]:
img = h5py.File(dir+'ovules_10k/N_294_final_crop_ds2_predictions.h5','r')
print(img.keys())
predictions_3dunet = img['predictions']
viewer = napari.view_image(predictions_3dunet,ndisplay=3,blending='additive')
#viewer.add_labels(trueseg)
napari.run()

<KeysViewHDF5 ['predictions']>


In [9]:
img1 = h5py.File(dir+'ovules_10k/PreProcessing/CHECKPOINT_DIR_ovules/N_294_final_crop_ds2_predictions.h5','r')
predictions_np_10k = np.array(img1['predictions'])
img2 = h5py.File(dir+'ovules/PreProcessing/CHECKPOINT_DIR_ovules/N_294_final_crop_ds2_predictions.h5','r')
predictions_np = np.array(img2['predictions'])
img3 = h5py.File(dir+'ovules_10k/N_294_final_crop_ds2_predictions.h5','r')
predictions_np_3dunet = np.array(img3['predictions'])

In [10]:
print((predictions_np_10k-predictions_np).sum())
print((predictions_np_10k-predictions_np_3dunet).sum())

-96648.31
35560.426


In [17]:
img1 = h5py.File(dir+'ovules_10k/N_294_final_crop_ds2.h5','r')
print(img1.keys())
raw = img1['raw']
img2 = h5py.File(dir+'ovules_10k/PreProcessing/CHECKPOINT_DIR_ovules/MultiCut/N_294_final_crop_ds2_predictions_multicut.h5','r')
print(img2.keys())
segmentation = img2['segmentation']

<KeysViewHDF5 ['label', 'label_with_ignore', 'raw']>
<KeysViewHDF5 ['segmentation']>


In [18]:
print(segmentation.shape)
n4=n0=0
segmentation_np = np.array(segmentation)
print((segmentation_np==4).sum())

(320, 960, 1000)
232978084


In [19]:
segmentation_np_t=np.where(segmentation_np==4, 0, segmentation_np)
print((segmentation_np_t==0).sum())

232978084


In [20]:
viewer = napari.view_image(raw,ndisplay=3,blending='additive')
viewer.add_labels(segmentation_np_t)
napari.run()

for ovules_10k:

2022-12-29 21:09:12,053 [MainThread] INFO UNet3DTrainer - Saving new best evaluation metric: 0.558465264602141

In [40]:
#Ovules (training data for confocal)
name = 'ovules'
img = h5py.File(dir+'ovules_10k/test/N_435_final_crop_ds2.h5','r')
#print(img4.keys())
raw = img['raw']
trueseg = img['label']
imglabelIgnore = img['label_with_ignore']

print('raw type and shape',type(raw),raw.shape)
print('truseg type and shape',type(trueseg),trueseg.shape)
# napari code - this will only run on laptop with napari installed
viewer = napari.view_image(raw,ndisplay=3,blending='additive')
viewer.add_labels(trueseg)
viewer.add_labels(imglabelIgnore)
napari.run()

raw type and shape <class 'h5py._hl.dataset.Dataset'> (388, 1101, 1110)
truseg type and shape <class 'h5py._hl.dataset.Dataset'> (388, 1101, 1110)


In [41]:
img1 = h5py.File(dir+'ovules_10k/test/N_435_final_crop_ds2.h5','r')
print(img1.keys())
raw = img1['raw']
img2 = h5py.File(dir+'ovules_10k/test/PreProcessing/CHECKPOINT_DIR_ovules/MultiCut/N_435_final_crop_ds2_predictions_multicut.h5','r')
print(img2.keys())
segmentation = img2['segmentation']
print(segmentation.shape)
print(segmentation[0][0][0])
lab = segmentation[0][0][0]
segmentation_np = np.array(segmentation)
print((segmentation_np==lab).sum())
segmentation_np_t=np.where(segmentation_np==lab, 0, segmentation_np)
print((segmentation_np_t==0).sum())
viewer = napari.view_image(raw,ndisplay=3,blending='additive')
viewer.add_labels(segmentation_np_t)
napari.run()

<KeysViewHDF5 ['label', 'label_with_ignore', 'raw']>
<KeysViewHDF5 ['segmentation']>
(388, 1101, 1110)
1
398331854
398331854


In [14]:
img = h5py.File(dir+'PreProcessing/Movie1_t00006_crop_gt.h5', 'r')
print(img.keys())
pre_raw = img['raw']
print('raw type and shape',type(pre_raw),pre_raw.shape)
viewer = napari.view_image(pre_raw,ndisplay=3,blending='additive')
napari.run()

<KeysViewHDF5 ['raw']>
raw type and shape <class 'h5py._hl.dataset.Dataset'> (486, 620, 1330)


In [27]:
img = h5py.File(dir+'CHECKPOINT_DIR/MutexWS/Movie1_t00006_crop_gt_predictions_gasp_mutex_watershed.h5', 'r')
print(img.keys())
seg = img['segmentation']
print('raw type and shape',type(seg),seg.shape)
viewer = napari.view_image(seg,ndisplay=3,blending='additive')
napari.run()

<KeysViewHDF5 ['segmentation']>
raw type and shape <class 'h5py._hl.dataset.Dataset'> (486, 620, 1330)


In [36]:
img = h5py.File(dir+'CHECKPOINT_DIR/GASP/Movie1_t00006_crop_gt_predictions_gasp_average.h5', 'r')
print(img.keys())
seg = img['segmentation']
print('raw type and shape',type(seg),seg.shape)
viewer = napari.view_image(seg,ndisplay=3,blending='additive')
napari.run()

<KeysViewHDF5 ['segmentation']>
raw type and shape <class 'h5py._hl.dataset.Dataset'> (486, 620, 1330)


In [37]:
img = h5py.File(dir+'CHECKPOINT_DIR/MultiCut/Movie1_t00006_crop_gt_predictions_multicut.h5', 'r')
print(img.keys())
gasp_seg = img['segmentation']
print('raw type and shape',type(gasp_seg),gasp_seg.shape)
viewer = napari.view_image(gasp_seg,ndisplay=3,blending='additive')
napari.run()

<KeysViewHDF5 ['segmentation']>
raw type and shape <class 'h5py._hl.dataset.Dataset'> (486, 620, 1330)


In [ ]:
img = h5py.File(dir+'CHECKPOINT_DIR/MultiCut/Movie1_t00006_crop_gt_predictions_multicut.h5', 'r')
print(img.keys())
seg = img['segmentation']
print('raw type and shape',type(seg),seg.shape)
viewer = napari.view_image(seg,ndisplay=3,blending='additive')
napari.run()

In [12]:
viewer = napari.view_image(trueseg,ndisplay=3,blending='additive')
viewer.add_labels(trueseg)
napari.run()

In [32]:
viewer = napari.view_image(imglabelIgnore,ndisplay=3,blending='additive')
viewer.add_labels(trueseg)
napari.run()

In [26]:
print(img)
print(img.keys())
print(img['raw'].attrs.keys())

<HDF5 file "Movie1_t00006_crop_gt.h5" (mode r)>
<KeysViewHDF5 ['label', 'raw']>
<KeysViewHDF5 []>


In [19]:
print(img)
print(img.keys())

<HDF5 file "Movie1_t00003_crop_gt.h5" (mode r)>
<KeysViewHDF5 ['label', 'raw', 'segmentation']>
